# **Notebooks sequence;)**
* Train-val split notebook [here](https://www.kaggle.com/chamecall/train-val-split).<br>
* Pretrain roberta-base on mlm with the competition data notebook [here](https://www.kaggle.com/chamecall/clrp-pretrain).<br>
* Finetune pretrained roberta-base on readability task notebook [here](https://www.kaggle.com/chamecall/clrp-finetune).<br>
* Inference model notebook [*CURRENT ONE*].<br>

In [ ]:

import os
from pathlib import Path


In [ ]:

os.chdir('../input/clrp-finetune/scripts')
exec(Path("imports.py").read_text())
exec(Path("config.py").read_text())
exec(Path("dataset.py").read_text())
exec(Path("model.py").read_text())
os.chdir('/kaggle/working')


In [ ]:

test_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
tokenizer = torch.load('../input/tokenizers/roberta-tokenizer.pt')
models_folder_path = Path('../input/clrp-finetune/models')
models_preds = []
n_models = 5

for model_num in range(n_models):
    print(f'Inference#{model_num+1}/{n_models}')
    test_ds = CLRPDataset(data=test_df, tokenizer=tokenizer, max_len=Config.max_len, is_test=True)
    test_sampler = SequentialSampler(test_ds)
    test_dataloader = DataLoader(test_ds, sampler = test_sampler, batch_size=Config.batch_size)
    model = torch.load(models_folder_path / f'best_model_{model_num}.pt').to(Config.device)

    all_preds = []
    model.eval()

    for step,batch in enumerate(test_dataloader):
        sent_id, mask = batch['input_ids'].to(Config.device), batch['attention_mask'].to(Config.device)
        with torch.no_grad():
            preds = model(sent_id, mask)
            all_preds += preds.flatten().cpu().tolist()
    
    models_preds.append(all_preds)

In [ ]:
models_preds = np.array(models_preds)
print(models_preds.shape)
print(models_preds)
all_preds = models_preds.mean(axis=0)*1.012045890799
print(all_preds.shape)
result_df = pd.DataFrame(
    {
        'id': test_df.id,
        'target': all_preds
    })


result_df.to_csv('submission.csv', index=False)
result_df.head(10)